In [0]:
import matplotlib.pyplot as plt
import numpy as np
import gzip
import pickle
import librosa

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Bidirectional
from keras.layers import ConvLSTM2D
from keras.utils import np_utils

In [11]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
pickle_off=gzip.open("drive/My Drive/COMP562/FinalProject/Old/HELP.data","rb")
originalData=pickle.load(pickle_off,encoding="latin1")

In [42]:
range(904)

range(0, 904)

In [0]:
sampling_rate=44100
nsamples=904;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
ntrain=600;
targets=42;
nfeatures=131*(nmfccs)

In [0]:
testingNSynth=np.zeros((nsamples,131,nmfccs+nchroma+6+1))
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.append(np.array(originalData[i]),np.zeros(500)), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.append(np.array(originalData[i]),np.zeros(500)), sr=sampling_rate,n_chroma=nchroma)
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.append(np.array(originalData[i]),np.zeros(500)),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    testingNSynth[i,:]=np.transpose(feature)

In [0]:
pickle_off=gzip.open("drive/My Drive/COMP562/FinalProject/TAL-NoiseMaker_onenote.data","rb")
emp=pickle.load(pickle_off)

In [0]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
ntrain=600;
targets=91;
nfeatures=131*(nmfccs)

In [136]:
X=np.zeros((nsamples,131,nmfccs+nchroma+6+1))
y=np.zeros((nsamples,targets))
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=np.transpose(feature)
    y[i,:]=([emp[i][1][j][1] for j in range(0,targets)])

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
numpy.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex
x_train=X[np.array(list(trainIndex),dtype=int),:]
y_train=y[np.array(list(trainIndex),dtype=int),:]
x_test=X[np.array(list(testIndex),dtype=int),:]
y_test=y[np.array(list(testIndex),dtype=int),:]

In [138]:
model=Sequential()
model.add(LSTM(24, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(targets, activation='relu'))
model.compile(optimizer='Nadam',
              loss='mse')
model.fit(x_train, y_train,
          epochs=40,
          batch_size=10)
score = model.evaluate(x_test, y_test, batch_size=100)
model.save('drive/My Drive/Comp562LSTM24Nadam241.h5')
print(np.sqrt(score))
#0.2948985627415366

Epoch 1/40
600/600 [==============================] - 7s 11ms/step - loss: 0.1787
Epoch 2/40
600/600 [==============================] - 4s 7ms/step - loss: 0.1064
Epoch 3/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0952
Epoch 4/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0911
Epoch 5/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0895
Epoch 6/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0876
Epoch 7/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0855
Epoch 8/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0848
Epoch 9/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0843
Epoch 10/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0842
Epoch 11/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0838
Epoch 12/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0836
Epoch 13/40
600/600 [===

In [0]:
results=model.predict(testingNSynth)

In [0]:
with open("drive/My Drive/COMP562/FinalProject/Old/NSynthEval_Tal.data", 'wb') as f:
  pickle.dump(results,f)
f.close()